In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
    !pip install synthetic-data-kit==0.0.3
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
    !pip install synthetic-data-kit==0.0.3

In [2]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## Generating synthetic dataset using meta synthetic dataset kit

our goal is to make the model understand "Attention is all you need" paper better.
and use  https://github.com/meta-llama/synthetic-data-kit to generate question and answer pairs which will be used for finetuning Llama

In [3]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-20 19:23:14 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-20 19:23:14 [__init__.py:239] Automatically detected platform cuda.


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

Unsloth: Using dtype = torch.float16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 89.39%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 7.19 GB. Also swap space = 0 GB.
vLLM STDOUT: INFO 05-20 19:23:31 [__init__.py:239] Automatically detected platform cuda.
vLLM STDOUT: INFO 05-20 19:23:40 [api_server.py:1043] vLLM API server version 0.8.5.post1
vLLM STDOUT: INFO 05-20 19:23:40 [api_server.py:1044] args: Namespace(subparser='serve', model_tag='unsloth/Llama-3.2-3B-Instruct', config='', host=None, port=8000, uvicorn_log_level='info', disable_uvicorn_access_log=False, allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, chat_template_content_format='auto', response_role='assi

## Generate question/answer pair

In [4]:
generator.prepare_qa_generation(
    output_folder = "data", # output location of synthetic data
    temperature = 0.7, # for more diverse datases
    top_p = 0.95,
    overlap = 64, # overlap portion during chunking
    max_generation_tokens = 512,
)

In [5]:
# check
!synthetic-data-kit system-check

 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1747769361, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-f258cded8f634038b49531458116961b', 'object': 'model_permission', 
'created': 1747769361, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


## Document Parsing(pdf in our case)

In [6]:
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://arxiv.org/html/1706.03762v7"

# Truncate document
filenames = generator.chunk_data("data/output/arxiv_org.txt")
print(len(filenames), filenames[:3])

⠋ Processing https://arxiv.org/html/1706.03762v7...
 Text successfully extracted to data/output/arxiv_org.txt
17 ['data/output/arxiv_org_0.txt', 'data/output/arxiv_org_1.txt', 'data/output/arxiv_org_2.txt']


We can see around 17 chunks of data.

Call synthetic-data-kit to create some pairs of data for 3 of the chunks

In [7]:
import time
# process 3 chunks
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 11 QA pairs total
Saving result to data/generated/arxiv_org_0_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/arxiv_org_0_qa_pairs.json
⠴ Generating qa content from data/output/arxiv_org_0.txt...
 Content saved to data/generated/arxiv_org_0_qa_pairs.json
Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 11 QA pairs total
Saving result to data/generated/arxiv_org_1_qa_pairs.json
Successfully wrote test file to data/generated/test_write.json
Successfully wrote result to data/generated/arxiv_org_1_qa_pairs.json
⠇ Generating qa content from data/output/arxiv_org_1.txt...
 Content saved to data/generated/arxiv_org_1_qa_pairs.json
Processing 1 chunks to generate QA pairs...
Batch processing complete.
Generated 13 QA pairs total
Saving result to data/generated/arxiv_org_2_qa_pairs.json
Successfully wrote test f

converting the generated data into QA format

In [8]:
qa_pairs_filenames = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_0_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_1_qa_pairs_ft.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to ft format with json 
storage...
 Converted to ft format and saved to data/final/arxiv_org_2_qa_pairs_ft.json


coverting qa pairs into alpaca format

In [9]:
qa_pairs_filenames_final = [
    f"data/generated/arxiv_org_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames_final:
  !synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    save-as {filename} --format alpaca

⠋ Converting data/generated/arxiv_org_0_qa_pairs.json to alpaca format with json
storage...
 Converted to alpaca format and saved to 
data/final/arxiv_org_0_qa_pairs_alpaca.json
⠋ Converting data/generated/arxiv_org_1_qa_pairs.json to alpaca format with json
storage...
 Converted to alpaca format and saved to 
data/final/arxiv_org_1_qa_pairs_alpaca.json
⠋ Converting data/generated/arxiv_org_2_qa_pairs.json to alpaca format with json
storage...
 Converted to alpaca format and saved to 
data/final/arxiv_org_2_qa_pairs_alpaca.json


load up data to see how dataset looks like

In [10]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/arxiv_org_{i}_qa_pairs_alpaca.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [11]:
dataset[0]

{'instruction': "Who is the author of the paper 'Attention Is All You Need'?",
 'input': '',
 'output': 'Ashish Vaswani'}

In [12]:
dataset[1]

{'instruction': "What is the main contribution of the paper 'Attention Is All You Need'?",
 'input': '',
 'output': 'The paper proposes a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.'}

In [13]:
dataset[-1]

{'instruction': 'What is the output of the softmax function in the attention function?',
 'input': '',
 'output': 'The weights on the values'}

In [14]:
len(dataset)

35

In [15]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
Server terminated gracefully.


In [16]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model = "unsloth/llama-3-8b-bnb-4bit"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit)

==((====))==  Unsloth 2025.5.6: Fast Llama patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [17]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias ='none',
    use_gradient_checkpointing = 'unsloth',
    random_state =123,
    use_rslora = False, #rank stabilized lora
    loftq_config = None
)

Unsloth 2025.5.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Formatting the dataset

In [18]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # add eos token

def format_entry(entry):
    instructions = entry["instruction"]
    inputs       = entry["input"]
    outputs      = entry["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # must add eos token
        text = prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [19]:
dataset = dataset.map(format_entry, batched = True,)
dataset

Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 35
})

## Train the model

In [20]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        push_to_hub=True,
        report_to = "tensorboard",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/35 [00:00<?, ? examples/s]

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 35 | Num Epochs = 15 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Step,Training Loss
1,3.332600
2,3.176200
3,3.352600
4,2.706000
5,2.663700
6,2.184400
7,1.576100
8,1.319700
9,1.065700
10,0.837000


## Inference

In [22]:
#for faster inference
FastLanguageModel.for_inference(model)

inputs = tokenizer(
[
    prompt.format(
        "What is attention?", # instruction
        " ", # input
        "", # output
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is attention?\n\n### Input:\n \n\n### Response:\nAttention is a mechanism that allows the model to attend to relevant parts of the input while ignoring irrelevant parts.<|end_of_text|>']